# Apresentação:

    **Escrever a apresentação**

In [21]:
import os
import json
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
from pyhdf.SD import SD, SDC
from processamento_dados import *

# Extraindo os dados:

    Temos dois códigos de LV3, a primeira versão é mais arcaico e a segundo versão é mais
    bem elaborado. A principio, vamos tentar extrair as tabelas utilizadas do LV3 mais bem
    elaborado e torcer para ele vir bem organizado.

In [2]:
# Defindo os diretórios:
dir1 = r'D:\Estudo\LASA\Dados\Bahia\L2_VFM'
dir2 = r'D:\Estudo\LASA\Dados\Bahia\L3_Total'

In [3]:
# Inervalo de Tempo:
ano_inicial = input('Data de Inicio(Ano): ')
ano_final = input('Data de Final(Ano): ')

    
if ano_inicial == '':
    ano_inicial = '2006'
    
elif int(ano_inicial) < 2006:
    print('Não existe valores anteriores a 2006')

    
if ano_final == '':
    ano_final = '2022'
    
elif int(ano_final) > 2022:
    print('Não existe valores maiores que 2022')

files = hdf_path(dir2, [ano_inicial,ano_final])

Data de Inicio(Ano):  
Data de Final(Ano):  


In [4]:
# importando informações:
info = json_reader('bahia_info.json')

In [5]:
# Variável contendo informações do produto de perfil de aerossóis troposféricos
info_produto = info['Produtos']['Tropospheric_Aerosol_Profile']

#files = [f.path for f in os.scandir(info_produto['Diretório'])]

niveis_altitude = SD(files[0], SDC.READ).select('Altitude_Midpoint').get()[0]
perfil_media_sazonal_anual = {}
perfil_media_sazonal = {}
dados = {}
for area, info_area in list(info['Áreas_estudo'].items()):
    # Cria um dicionário vazio para cada área
    perfil_media_sazonal_anual[area] = {}
    perfil_media_sazonal[area] = {}
    dados[area] = {}
    for variavel, info_variavel in info_produto["Variáveis"].items():
        # Cria uma chave no dicionário para cada variável
        dados[area][variavel] = {}
        coordenadas = info_area['Coordenadas']
        for file in files:
            dado = SD(file, SDC.READ)
            # Abre cada arquivo listado
            # Seleciona dados de cada variável fazendo corte de área
            dado = selecao_area(dado, coordenadas, variavel)
            dado = controle_qualidade(dado, info_variavel['Limites_detecção'])
            # Adicionar mês ao dicionário apenas quando 80% dos dados passam no CQ
            if porcentagem_valida(dado) >= 0.8:
                dados[area][variavel][file[-12:-5]] = calc_media_espacial(dado)

        if variavel == 'Aerosol_Type':
            # Transforma array contendo classificação dos aerossóis em dataframe com colunas correspondendo a cada tipo de aerossol
            dados[area][variavel] = convert_dicionario_2d_dataframe(dados[area][variavel], info_variavel)
            # Salvando df com médias sazonais para cada ano e tipo de aerossol na chave da variável de detecção de aerossóis por tipo
            perfil_media_sazonal_anual[area][variavel] = calc_media_sazonal_anual_2d(dados[area][variavel])
            perfil_media_sazonal[area][variavel] = calc_media_sazonal_2d(perfil_media_sazonal_anual[area][variavel])
        else:
            # Salvando df com as médias sazonais para cada ano de cada variável de perfil único na chave da variável
            perfil_media_sazonal_anual[area][variavel] = calc_media_sazonal_anual_1d(dados[area][variavel])
            # Salvando df com as médias sazonais do período para cada variável de perfil único na chave da variável
            perfil_media_sazonal[area][variavel] = calc_media_sazonal_1d(perfil_media_sazonal_anual[area][variavel])
        
        if 'Ext' not in variavel:
            # Calcula a soma das detecções para a coluna
            dados[area][variavel] = calc_soma_deteccoes_coluna(dados[area][variavel])
            # Tranformando dicionário em df
            dados[area][variavel] = convert_dicionario_1d_dataframe(dados[area][variavel])
            dados[area][variavel] = preenchendo_dados(dados[area][variavel])

    dados[area]['AOD'] = pd.concat({variavel: valor for variavel, valor in dados[area].items() if 'AOD' in variavel}, axis=1).droplevel(1, axis=1)

C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:152: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:152: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:152: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:152: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:152: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.nanmean(dado, axis=0), axis=0)
C:\Users\amori\JupyterNotebook\OneDrive\LASA\Experimentos\processamento_dados.py:152: RuntimeWarning: Mean of empty slic

# Transferência dos dados:

    Algumas tabelas estão com a data como index. Não queremos isso dessa forma, porque
    na hora de passar para o SGBD pode ser que percamos essa informação. Então vamos
    tratar esses dados para extrair essa informação da melhor forma possivel.

In [14]:
import mysql.connector
from conection import *

In [15]:
# Conectando-se ao SGBD:
mydb = mysql.connector.connect(
  host="localhost",
  user="bd_project",
  password="*****"
)

if mydb.is_connected() == True:
    print('Conectado ao SGBD')

Conectado ao SGBD


In [16]:
# Instanciando o cursor para transferência de informação:
cursor = mydb.cursor()

In [22]:
# Criando Schema:
inicio = time()
for area in tqdm(dados.keys()):
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {area}")
    cursor.execute(f"USE {area}")

    # Criando tabelas de Coeficiente de Extinção:
    for key in dados[area].keys():
        if 'Extinction_Coefficient' in key:
            colunas = ', '.join([f'`{col}` FLOAT' for col in dados[area][key]])
            cursor.execute(
                f'''
                CREATE TABLE IF NOT EXISTS {key} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    {colunas}
                );
                '''
            )
            
            # Transferindo dados:
            dataframe = pd.DataFrame(dados[area][key])
            # Ajustando valores nan para serem inseridos na tabela:
            dataframe.replace({'nan': None, np.nan: None}, inplace=True)
        
            # Extraindo as colunas como string:
            colunas = ', '.join([f'`{col}`' for col in dataframe.columns])
        
        
            # Preparando os valores para inserção
            values = [tuple(row) for row in dataframe.to_numpy()]
        
            # Construindo a instrução SQL com %s para cada valor
            insert = f"INSERT INTO {key} ({colunas}) VALUES ({', '.join(['%s'] * len(dataframe.columns))})"
        
            # Executando a inserção em lote
            cursor.executemany(insert, values)
            mydb.commit()
        
        elif key == "Aerosol_Type":
            # Construindo Tabelas:
            cursor.execute(
                f'''
                CREATE TABLE IF NOT EXISTS {key} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    Data VARCHAR(20),
                    Marinho_Limpo FLOAT,
                    Poeira FLOAT,
                    Poluicao_Continental_Fumaca FLOAT,
                    Continental_Limpo FLOAT,
                    Poeira_Poluida FLOAT,
                    Fumaca_Elevada FLOAT,
                    Poeira_Marinha FLOAT
                );
                '''
            )
    
            # Inserindo dados:
            dataframe = pd.DataFrame(dados[area][key]).reset_index().rename(columns={'index': 'Data'})
            dataframe.replace({np.nan: None}, inplace=True)
            
            for index, row in dataframe.iterrows():
                insert = '''INSERT INTO Aerosol_Type (Data, Marinho_Limpo, Poeira,
                Poluicao_Continental_Fumaca,Continental_Limpo,Poeira_Poluida,Fumaca_Elevada,Poeira_Marinha)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'''
                val = (row['Data'],row['Marinho Limpo'],row['Poeira'],
                       row['Poluição Continental/Fumaça'],row['Continental Limpo'],row['Poeira Poluída'],
                       row['Fumaça Elevada'],row['Poeira Marinha'])
                cursor.execute(insert, val)
                mydb.commit()

        elif key == "AOD":
            # Construindo Tabelas:
            cursor.execute(
                f'''
                CREATE TABLE IF NOT EXISTS {key} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    Data VARCHAR(20),
                    AOD_Mean FLOAT,
                    AOD_Mean_Dust FLOAT,
                    AOD_Mean_Elevated_Smoke FLOAT,
                    AOD_Mean_Polluted_Dust FLOAT
                );
                '''
            )
    
            # Inserindo dados:
            dataframe = pd.DataFrame(dados[area][key]).reset_index().rename(columns={'index': 'Data'})
            dataframe.replace({np.nan: None}, inplace=True)
    
            for index, row in dataframe.iterrows():
                insert = '''INSERT INTO AOD (Data, AOD_Mean, AOD_Mean_Dust,AOD_Mean_Elevated_Smoke,AOD_Mean_Polluted_Dust)
                VALUES (%s,%s,%s,%s,%s)'''
                val = (row['Data'],row['AOD_Mean'],row['AOD_Mean_Dust'],
                       row['AOD_Mean_Elevated_Smoke'],row['AOD_Mean_Polluted_Dust'])
                cursor.execute(insert, val)
                mydb.commit()
    
        
        else:
            cursor.execute(
                f'''
                CREATE TABLE IF NOT EXISTS {key} (
                Data VARCHAR(20),
                Valores FLOAT
                )
                '''
            )

            # Transferindo dados:
            dataframe = pd.DataFrame(dados[area][key])
            dataframe.rename(columns={0:'Valores'},inplace=True)
            # Ajustando valores nan para serem inseridos na tabela:
            dataframe.replace({'nan': None, np.nan: None}, inplace=True)
        
            # Extraindo as colunas como string:
            colunas = ', '.join([f'`{col}`' for col in dataframe.columns])
        
        
            # Preparando os valores para inserção
            values = [tuple(row) for row in dataframe.to_numpy()]
        
            # Construindo a instrução SQL com %s para cada valor
            insert = f"INSERT INTO {key} ({colunas}) VALUES ({', '.join(['%s'] * len(dataframe.columns))})"
        
            # Executando a inserção em lote
            cursor.executemany(insert, values)
            mydb.commit()


            
final = time()
print('Processo Concluido:')
print('Tempo de processamento:', int(final - inicio), 'segundos')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:23<00:00,  4.63s/it]

Processo Concluido:
Tempo de processamento: 23 segundos
